In [2]:
import numpy as np
import mlflow
import os

In [3]:
# Establecer la URI de la base de datos SQLite y configuración de MLflow
os.environ['MLFLOW_TRACKING_URI'] = 'sqlite:///mlruns.db'
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("regresion_lineal")

<Experiment: artifact_location='mlflow-artifacts:/3', creation_time=1713193978222, experiment_id='3', last_update_time=1713193978222, lifecycle_stage='active', name='regresion_lineal', tags={}>

# Con 19 Variables sin offdrb

In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
import mlflow
import preprocesamiento_datos

# Obtener los datos preprocesados
X_train, X_val, X_test, y_train, y_val, y_test,RANDOM_STATE = preprocesamiento_datos.preprocesamiento(False,['Mins', 'Goals', 'Assists', 'SpG', 'PS%', 'MotM', 'KeyP', 'AvgP', 'ThrB', 'Titularidades', 'Equipo_pos', '1_año_anterior', '2_año_anterior', '3_año_anterior', '4_año_anterior', '5_año_anterior', 'Equipo_FC Barcelona', 'Equipo_Real Madrid', 'nationality_Germany'], True)
# Crear el modelo de regresión lineal
model = LinearRegression()

# Entrenar el modelo con los datos de entrenamiento
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de validación
y_val_pred = model.predict(X_val)

# Calcular el error absoluto medio en el conjunto de validación
val_mae = mean_absolute_error(y_val, y_val_pred)

# Iniciar el registro de la ejecución en MLflow
with mlflow.start_run(run_name="MODELO FINAL ESCALADO"):
    # Registrar el error absoluto medio en el conjunto de validación en MLflow
    mlflow.log_metric("val_mae", val_mae)
    from sklearn.metrics import r2_score
    import numpy as np
    # Número de muestras en el conjunto de prueba
    n_samples = len(y_test)
    
    # Entrenar el modelo final con todos los datos de entrenamiento y validación
    X_train_final = np.concatenate((X_train, X_val))
    y_train_final = np.concatenate((y_train, y_val))
    model.fit(X_train_final, y_train_final)

    # Realizar predicciones en el conjunto de prueba
    y_test_pred = model.predict(X_test)
    
    # Calcular el error absoluto medio en el conjunto de prueba
    test_mae = mean_absolute_error(y_test, y_test_pred)

    # Registrar el error absoluto medio en el conjunto de prueba en MLflow
    mlflow.log_metric("test_mae", test_mae)
    print("Test MAE:", test_mae)
    

Test MAE: 2658272.751551002


# 20 Variables Con OffDRB

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
import mlflow
import preprocesamiento_datos

# Obtener los datos preprocesados
X_train, X_val, X_test, y_train, y_val, y_test,RANDOM_STATE = preprocesamiento_datos.preprocesamiento(False,['Mins', 'Goals', 'Assists', 'SpG', 'PS%', 'MotM', 'KeyP', 'OffDrb', 'AvgP', 'ThrB', 'Titularidades', 'Equipo_pos', '1_año_anterior', '2_año_anterior', '3_año_anterior', '4_año_anterior', '5_año_anterior', 'Equipo_FC Barcelona', 'Equipo_Real Madrid', 'nationality_Germany'], True)
# Crear el modelo de regresión lineal
model = LinearRegression()

# Entrenar el modelo con los datos de entrenamiento
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de validación
y_val_pred = model.predict(X_val)

# Calcular el error absoluto medio en el conjunto de validación
val_mae = mean_absolute_error(y_val, y_val_pred)

# Iniciar el registro de la ejecución en MLflow
with mlflow.start_run(run_name="MODELO FINAL ESCALADO"):
    # Registrar el error absoluto medio en el conjunto de validación en MLflow
    mlflow.log_metric("val_mae", val_mae)
    from sklearn.metrics import r2_score
    import numpy as np

    # Número de muestras en el conjunto de prueba
    n_samples = len(y_test)
    
    # Entrenar el modelo final con todos los datos de entrenamiento y validación
    X_train_final = np.concatenate((X_train, X_val))
    y_train_final = np.concatenate((y_train, y_val))
    model.fit(X_train_final, y_train_final)

    # Realizar predicciones en el conjunto de prueba
    y_test_pred = model.predict(X_test)
    
    # Calcular el error absoluto medio en el conjunto de prueba
    test_mae = mean_absolute_error(y_test, y_test_pred)

    # Registrar el error absoluto medio en el conjunto de prueba en MLflow
    mlflow.log_metric("test_mae", test_mae)
    print("Test MAE:", test_mae)

Test MAE: 2564070.2922655987


# Modelo Completo

In [21]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
import mlflow
import preprocesamiento_datos

# Obtener los datos preprocesados
X_train, X_val, X_test, y_train, y_val, y_test = preprocesamiento_datos.preprocesamiento(False,[], False)
# Crear el modelo de regresión lineal
model = LinearRegression()

# Entrenar el modelo con los datos de entrenamiento
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de validación
y_val_pred = model.predict(X_val)

# Calcular el error absoluto medio en el conjunto de validación
val_mae = mean_absolute_error(y_val, y_val_pred)

# Iniciar el registro de la ejecución en MLflow
with mlflow.start_run(run_name="MODELO CON TODAS LAS VARIABLES"):
    # Registrar el error absoluto medio en el conjunto de validación en MLflow
    mlflow.log_metric("val_mae", val_mae)

    # Entrenar el modelo final con todos los datos de entrenamiento y validación
    X_train_final = np.concatenate((X_train, X_val))
    y_train_final = np.concatenate((y_train, y_val))
    model.fit(X_train_final, y_train_final)

    # Realizar predicciones en el conjunto de prueba
    y_test_pred = model.predict(X_test)
    
    # Calcular el error absoluto medio en el conjunto de prueba
    test_mae = mean_absolute_error(y_test, y_test_pred)

    # Registrar el error absoluto medio en el conjunto de prueba en MLflow
    mlflow.log_metric("test_mae", test_mae)
    print("Test MAE:", test_mae)

Archivo Datos_la_liga_preparados_entrenamiento.parquet guardado en: ../Downloads\Datos_la_liga_preparados_entrenamiento.parquet
Test MAE: 2865652.999008513


C:\Users\Usuario\Documents\Alvaro\Carrera\2-Segundo\Primer cuatrimetre\Fundamentos de la Inteligencia Artificial\PycharmProjects\proyecto\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
